# Génération du dataset pour l'analyse de données

Les données issues du scraping de l'API IDF Mobilités ne constituent pas un jeu de données en tant que tel. Le rôle de ce notebook est de passer du des données brutes à un jeu de données exploitable par la suite.

In [1]:
# On utilise cudf.pandas pour accélérer les opérations Pandas sur GPU, optionnel

!pip install \
  --extra-index-url=https://pypi.nvidia.com \
  cudf-cu12==24.12.* \
  dask-cudf-cu12==24.12.* \
  cuml-cu12==24.12.* \
  cugraph-cu12==24.12.*

%load_ext cudf.pandas

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [2]:
!pip install -r requirements.txt

In [3]:

import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

Pour avoir plus de flexibilité pour mener notre analyse, nous ne travaillerons pas seulement avec un dataset combinant les données sur les perturbations et les lignes affectées, nous garderons aussi de côté les données propres aux perturbations et aux lignes séparées. Aussi, du fait de perturbations pouvant être de longue durée (travaux...), nos données brutes comportent de nombreux doublons que nous allons devoir traiter.

Nous traiterons les données de sorte à n'avoir plus qu'à créer nos dataframes à partir des objets que nous constitutions ici. Créer des dataframes plus rudimentaires puis les raffiner s'avère être un processus très complexe du fait d'objets imbriqués et de listes de longueurs variables dans nos données brutes.

In [4]:
import os
import json
import pandas as pd
import s3fs

KEY = os.environ.get("MINIO_KEY")
SECRET = os.environ.get("MINIO_SECRET_KEY")

fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
    key=KEY,
    secret=SECRET
)

paths = fs.ls(ROOT)
paths = [p for p in paths if p.endswith('.json')]

all_results = []
all_disruptions = []
all_objects = []

results_data = []
disruptions_data = []
objects_data = []
objects_disruptions_data = []

seen_results = set()              # last_updated
seen_disruptions = set()          # (disruption_id, begin, end)
seen_objects = set()              # object_id
seen_objects_disruptions = set()  # (object_id, disruption_id)

for file_path in paths:
    try:
        with fs.open(file_path, "r", encoding="ascii") as f:
            data = json.loads(f.read())

            last_updated = data.get("lastUpdatedDate", None)

            all_results.append(last_updated)
            if last_updated not in seen_results:
                results_data.append(last_updated)

            for d in data.get("disruptions", []):
                disruption_id = d.get("id")
                all_disruptions.append(disruption_id)
                for p in d.get("applicationPeriods", []):
                    key = (disruption_id, p.get("begin"), p.get("end"))
                    if key not in seen_disruptions:
                        seen_disruptions.add(key)
                        disruptions_data.append({
                            "disruption_id": disruption_id,
                            "begin": p.get("begin"),
                            "end": p.get("end"),
                            "lastUpdate": d.get("lastUpdate"),
                            "cause": d.get("cause"),
                            "severity": d.get("severity"),
                            "title": d.get("title"),
                            "message": d.get("message"),
                            "file_lastUpdatedDate": last_updated,
                        })

            for l in data.get("lines", []):
                line_info = {
                    "line_id": l.get("id"),
                    "line_name": l.get("name"),
                    "line_shortName": l.get("shortName"),
                    "line_mode": l.get("mode"),
                    "line_networkId": l.get("networkId"),
                    "file_lastUpdatedDate": last_updated,
                }
                for o in l.get("impactedObjects", []):
                    object_id = o.get("id")
                    all_objects.append(object_id)
                    if object_id not in seen_objects:
                        seen_objects.add(object_id)
                        objects_data.append({
                            **line_info,
                            "object_id": object_id,
                            "object_name": o.get("name"),
                            "object_type": o.get("type"),
                        })

                    for disruption_id in o.get("disruptionIds", []):
                        key = (object_id, disruption_id)
                        if key not in seen_objects_disruptions:
                            seen_objects_disruptions.add(key)
                            objects_disruptions_data.append({
                                **line_info,
                                "object_id": object_id,
                                "object_name": o.get("name"),
                                "object_type": o.get("type"),
                                "disruption_id": disruption_id,
                            })
    except Exception as e:
        print("Error occurred with file:", file_path)
        raise e

df_disruptions = pd.DataFrame(disruptions_data)
df_objects = pd.DataFrame(objects_data)
df_objects_disruptions = pd.DataFrame(objects_disruptions_data)

NameError: name 'ROOT' is not defined

In [20]:
print("Total results (all):", len(all_results))
print("Total disruptions (all):", len(all_disruptions))
print("Total objects (all):", len(all_objects))

print("#####################")

print("Total results (no duplicates):", len(results_data))
print("Total disruptions (no duplicates):", len(disruptions_data))
print("Total objects (no duplicates):", len(objects_data))
print("Total objects/disruptions tuples", len(objects_disruptions_data))

Total results (all): 2357
Total disruptions (all): 1724401
Total objects (all): 5476570
#####################
Total results (no duplicates): 2357
Total disruptions (no duplicates): 30177
Total objects (no duplicates): 7570
Total objects/disruptions tuples 102807


On comprend ici que l'API IDF Mobilités est mise à jour plus régulièrement que notre fréquence de scraping (aucun doublon dans les résultats des appels API). Cela veut dire qu'il n'est pas impossible que nous ayons manqué des perturbations de très courte durée sur la période considérée. Nous garderons cela en tête pour l'analyse des données.

La déduplication a toutefois été très importante pour les données sur les perturbations et sur les lignes, ce qui était attendu. Avec 19757 perturbations différentes dans notre jeu de données établi sur 3 semaines. Nous avons assez de données pour faire une analyse intéressante, bien que l'idéal serait de produire un outil permettant une analyse continue et automatisée des perturbations fournies par l'API. Avant tout, enregistrons ce jeu de données fraîchement généré.

In [ ]:
df_disruptions.to_feather("data/disruptions.feather")
df_objects.to_feather("data/objects.feather")
df_objects_disruptions.to_feather("data/objects_disruptions.feather")

/opt/conda/lib/python3.12/site-packages/cudf/io/feather.py:26: UserWarning: Using CPU via PyArrow to write Feather dataset, this may be GPU accelerated in the future
  warnings.warn(
